In [1]:
import rospy
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
import os, shutil

from ipywidgets import FloatProgress
from IPython.display import display
import time

In [2]:
vc = VirtualCoach(environment='local', storage_username='nrpuser')
# Type "password"

INFO: [2018-10-08 11:18:10,107 - Configuration] Loading configuration file config.json
INFO: [2018-10-08 11:18:10,108 - Configuration] Using user specified environment: local
INFO: [2018-10-08 11:18:10,110 - VirtualCoach] Logging into the Storage Server as: nrpuser
········
INFO: [2018-10-08 11:18:12,818 - VirtualCoach] Ready.


In [4]:
configName = "fable_cerebellum_recurrent_torque"

In [217]:
folder = "/home/dtu-neurorobotics/Documents/workspace/Projects/DTU-Pavia_Cerebellum_Carlos_recurrent_torque/cereb_tuning/results/"
test = "3"
folder = folder+test
try:
    file_list = os.listdir(folder)
except Exception as e:
    print("Creating experiment folder: " + test)
    os.mkdir(folder)
    file_list = os.listdir(folder)
    
    
        
if len(file_list):
    print("Going to delete everything here:")
    #print(file_list)
    answer = raw_input("You sure? (y/n)")

    if answer == "y":
        
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path): 
                    shutil.rmtree(file_path)

            except Exception as e:
                print(e)

        print("Success")
    else:
        print("Cancelled")
else:
    print("Empty directory")

Going to delete everything here:
You sure? (y/n)y
Success


In [218]:
experiment = vc.clone_experiment_to_storage(configName)
print(experiment)

INFO: [2018-10-09 13:48:04,540 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-09 13:48:04,581 - VirtualCoach] Experiment "fable_cerebellum_recurrent_torque" cloned successfully
fable_manipulation_dtu_spiking_cereb_recurrent_torque_0


In [219]:
sim = vc.launch_experiment(experiment)

INFO: [2018-10-09 13:48:05,586 - VirtualCoach] Preparing to launch fable_manipulation_dtu_spiking_cereb_recurrent_torque_0.
INFO: [2018-10-09 13:48:05,587 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-09 13:48:05,593 - VirtualCoach] Retrieving list of available servers.
[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
INFO: [2018-10-09 13:48:05,599 - Simulation] Attempting to launch fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 on localhost.
INFO: [2018-10-09 13:48:20,877 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Simulation Successfully Created.
INFO: [2018-10-09 13:48:20,881 - Simulation (fable_manipu

In [220]:
run_time = 15

sim.start()

walltime_start = time.time()

time_started = rospy.Time.now()
delta_t = rospy.Time.now() - time_started

print("\nStart delta_t: " + str(delta_t.to_sec()) + " (Should be 0)")

f = FloatProgress(min=0, max=run_time) # instantiate the bar

print("Simulating " + str(run_time) + " seconds. Simulation Progress:")
display(f) # display the bar

offset = rospy.Duration(0)
first_run = True
try:
    while delta_t.to_sec() < run_time:
        delta_t = rospy.Time.now() - time_started - offset
        
        if first_run:
            if delta_t.to_sec() < 0:
                offset = delta_t
            first_runt = False
            
        #print delta_t.to_sec(), "(Time offset: ",offset.to_sec(), ")"
        f.value = delta_t.to_sec()
        time.sleep(3)
except KeyboardInterrupt:
    pass

f.bar_style='danger'

sim.pause()
#sim.stop()
walltime_end = time.time()
print("\nSimulation paused at: " + str(delta_t.to_sec()) + ", executed in " + str(walltime_end - walltime_start) + " sec")

INFO: [2018-10-09 13:48:21,878 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Attempting to transition to state: started
INFO: [2018-10-09 13:48:21,883 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Simulation state: started

Start delta_t: 0.0 (Should be 0)
Simulating 15 seconds. Simulation Progress:


FloatProgress(value=0.0, max=15.0)

INFO: [2018-10-09 13:50:31,007 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Attempting to transition to state: paused
INFO: [2018-10-09 13:50:31,022 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Simulation state: paused

Simulation paused at: 15.191, executed in 129.139355898 sec


In [221]:
sim.print_csv_file_names()
sim.save_csv()

###csv_data = {'planner_TF' : sim.get_csv_data('joint_info.csv'), 
###            'controller_TF': sim.get_csv_data('command.csv')}

csv_directory = "/home/dtu-neurorobotics/.opt/nrpStorage/" + experiment +"/"

file_list = os.listdir(csv_directory)
for i in file_list:
    if os.path.isdir(csv_directory+i) and "resources" not in i:
        shutil.copyfile(csv_directory+i+"/"+"joint_info.csv", folder+"/"+"joint_info.csv")
        shutil.copyfile(csv_directory+i+"/"+"command.csv", folder+"/"+"command.csv")

###print "\nLaste entry in CSV at time :", csv_data['planner_TF'][-1][0]
###print ""

sim.stop()


INFO: [2018-10-09 13:50:37,585 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Attempting to retrieve recorders' csv data
[u'command.csv', u'joint_info.csv']
INFO: [2018-10-09 13:50:37,661 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Saved CSV data
INFO: [2018-10-09 13:50:37,664 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Attempting to transition to state: stopped
INFO: [2018-10-09 13:50:37,763 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Simulation has been stopped.
INFO: [2018-10-09 13:50:38,675 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Simulation state: stopped


In [222]:
sim.stop()
vc.delete_cloned_experiment(experiment)

INFO: [2018-10-09 13:50:39,866 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Attempting to transition to state: stopped
INFO: [2018-10-09 13:50:40,875 - Simulation (fable_manipulation_dtu_spiking_cereb_recurrent_torque_0 - localhost #45)] Simulation state: stopped
INFO: [2018-10-09 13:50:40,876 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-09 13:50:40,887 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_recurrent_torque_0" deleted successfully


In [223]:
vc.print_cloned_experiments()

INFO: [2018-10-09 13:50:45,476 - VirtualCoach] Retrieving list of experiments.
+------+
| Name |
+======+
+------+


In [334]:
sim.stop()

INFO: [2018-09-26 11:38:18,299 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #94)] Attempting to transition to state: stopped
ERROR: [2018-09-26 11:38:18,303 - bbp_client.oidc.client] Failed to retrieve data: Internal Server Error


HTTPError: HTTP Error 500: Invalid response 500.

In [123]:
sim.print_csv_file_names()
sim.save_csv()

###csv_data = {'planner_TF' : sim.get_csv_data('joint_info.csv'), 
###            'controller_TF': sim.get_csv_data('command.csv')}

csv_directory = "/home/dtu-neurorobotics/.opt/nrpStorage/" + experiment +"/"

file_list = os.listdir(csv_directory)
for i in file_list:
    if os.path.isdir(csv_directory+i):
        shutil.copyfile(csv_directory+i+"/"+"joint_info.csv", folder+"/"+"joint_info.csv")
        shutil.copyfile(csv_directory+i+"/"+"command.csv", folder+"/"+"command.csv")

###print "\nLaste entry in CSV at time :", csv_data['planner_TF'][-1][0]
###print ""

sim.stop()


INFO: [2018-09-18 12:41:29,073 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #13)] Attempting to retrieve recorders' csv data
[]
INFO: [2018-09-18 12:41:29,091 - Simulation (fable_manipulation_dtu_spiking_cereb_v2.0_0 - localhost #13)] Saved CSV data


IOError: [Errno 2] No such file or directory: '/home/dtu-neurorobotics/.opt/nrpStorage/fable_manipulation_dtu_spiking_cereb_v2.0_0/csv_records_2018-09-18_12-41-29/joint_info.csv'

In [126]:
vc.delete_cloned_experiment(experiment)

INFO: [2018-09-18 12:44:55,080 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-09-18 12:44:55,094 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_v2.0_0" deleted successfully


In [21]:
vc.print_cloned_experiments()

INFO: [2018-10-08 11:02:25,443 - VirtualCoach] Retrieving list of experiments.
+------+
| Name |
+======+
+------+


In [20]:
vc.delete_cloned_experiment("fable_manipulation_dtu_spiking_cereb_recurrent_torque_1")

INFO: [2018-10-08 11:02:23,345 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-10-08 11:02:23,368 - VirtualCoach] Experiment "fable_manipulation_dtu_spiking_cereb_recurrent_torque_1" deleted successfully
